In [ ]:


import pandas as pd
import dash 
from dash import dcc, html, Input, Output
import plotly.express as px


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
data = {
    'Year': [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022],
    'Winner': ['Uruguay', 'Italy', 'Italy', 'Uruguay', 'Germany', 'Brazil', 'Brazil', 'England', 'Brazil', 'Germany', 'Argentina', 'Italy', 'Argentina', 'Germany', 'Brazil', 'France', 'Brazil', 'Italy', 'Spain', 'Germany', 'France', 'Argentina'],
    'Runner-up': ['Argentina', 'Czechoslovakia', 'Hungary', 'Brazil', 'Hungary', 'Suecia', 'Yugoslavia', 'Germany', 'Italy', 'Netherlands', 'Netherlands', 'Germany', 'Germany', 'Argentina', 'Brazil', 'Brazil', 'Germany', 'France', 'Netherlands', 'Argentina', 'Croatia', 'France']
}

df=pd.DataFrame(data)


win_counts=df['Winner'].value_counts().reset_index()
win_counts.columns=['Country','Wins']

app=dash.Dash(__name__)

app.layout=html.Div([
    html.H1("FIFA World Cup Winners Dashboard"),

    dcc.Graph(id='world-map'),

    html.Label("Select a country to view wins: "),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df['Winner'].unique()],
        value='Argentina'
    ),
    html.Div(id='country-info'),

    html.Label("Select a year to view winner & runner-up: "),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df['Year']],
        value=2022
    ),
    html.Div(id='year-info')
])

@app.callback(
    Output('world-map', 'figure'),
    Input('country-dropdown', 'value')
)
def update_map(selected_country):
    fig = px.choropleth(
        win_counts,  # Data containing country wins
        locations='Country',
        locationmode='country names',
        color='Wins',
        hover_name='Country',
        title="FIFA World Cup Wins by Country",
        color_continuous_scale="Blues"
    )
    return fig

@app.callback(
    Output('country-info','children'),
    Input('country-dropdown','value')
)

def display_country_info(country):
    wins = (df['Winner'] == country).sum()
    return f"{country} has won {wins} times." if wins > 0 else f"{country} has not won any World Cups."

@app.callback(
    Output('year-info','children'),
    Input('year-dropdown','value')
)

def display_year_info(year):
    row=df[df['Year']==year]
    
    return f"{year}: Winner - {row['Winner'].values[0]}, Runner-up - {row['Runner-up'].values[0]}" 

if __name__ == '__main__':
    app.run(debug=True, port=8052)  